In [1]:
import pandas as pd
import numpy as np
import re
import random
import spacy
nlp = spacy.load("fr_core_news_lg")
import pickle
import itertools
from tqdm.notebook import tqdm

# X pretreatement functions

In [2]:
df = pd.read_excel("../algorel_fullDb_190223.xlsx")
df.head()

,TaxonomyPath,TaxonomyCode,Texts,ProductUid
0,Outillage - Droguerie - Quincaillerie/Drogueri...,OU_4/18/3/,PROPFEU CREME LUSTRANTE boite 220 ml\nwww.gebs...,501064435
1,Chauffage/Fumisterie/Accessoires de fumisterie,CH_2/11/6/,TRESSE EXTENSIBLE EN FIBRE DE VERRE (Diam 9/10...,501064447
2,Chauffage/Fumisterie/Accessoires de fumisterie,CH_2/11/6/,TRESSE EXTENSIBLE EN FIBRE DE VERRE (Diam 11/1...,501064448
3,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/2/9/1/,ROULEAU ATELIER WATERPROOF Rouleau 38 mm x 25 ...,501064453
4,Outillage - Droguerie - Quincaillerie/Drogueri...,OU_4/18/4/4/,BANDE D'ETANCHEITE couleur aluminium rouleau ...,501064455


In [3]:
df.isna().sum()

TaxonomyPath    6505
TaxonomyCode    6505
Texts              0
ProductUid         0
dtype: int64

<p style='background-color:red; '> We drop the 6505 items without Taxonomy </p>


In [4]:
df.dropna(inplace=True)
print(len(df))
df.isna().sum()

95855


TaxonomyPath    0
TaxonomyCode    0
Texts           0
ProductUid      0
dtype: int64

In [5]:
keyword = "nicoll"
db = df.loc[df.Texts.str.lower().str.contains(keyword),:]
len(db)

1291

# Compare Rouge and Levenshtein v2

In [65]:
df_metrics = pd.read_excel("./"+keyword+"_consolidated.xlsx")

In [66]:
df_metrics

,Unnamed: 0,Rouge,In_A_not_B,In_B_not_A,ProductUid_A,ProductUid_B,variant,rank_levenshtein,Levenshtein
0,0,0.991561,"TF18, 0.081","0.046, TF188MF",501059169,501059172,"(501059169, 501059172)",6608,0.833333
1,1,0.989691,TD100S,"prem, TD100PS, Avec",501059150,501059149,"(501059149, 501059150)",1408,0.931507
2,2,0.989429,"0.109, d.50, TJ18","TJ38, d.50/32, 0.079",501059211,501059230,"(501059211, 501059230)",691,0.952381
3,3,0.989305,TD80S,"prem, TD80PS, Avec",501059160,501059159,"(501059159, 501059160)",1485,0.929577
4,4,0.989154,"d.32, 0.075, AF14","50, 0.170, AJ14, d.50",501056883,501056936,"(501056883, 501056936)",1651,0.925926
...,...,...,...,...,...,...,...,...,...
832690,832690,0.000000,not match,not match,501058874,501058780,"(501058780, 501058874)",72224,0.481481
832691,832691,0.000000,not match,not match,501057108,501058780,"(501057108, 501058780)",77535,0.474576
832692,832692,0.000000,not match,not match,501056842,501058780,"(501056842, 501058780)",718514,0.264706
832693,832693,0.000000,not match,not match,501057064,501058780,"(501057064, 501058780)",604492,0.294118


In [67]:
def visualize_variants(df,variants):
    """
    Returns the description stored in df corresponding to all ProtuctUid stored in variant
    """
#     return df.loc[df.ProductUid==variant[0],"Texts"].iloc[0],df.loc[df.ProductUid==variant[1],"Texts"].iloc[0]
    return tuple(df.loc[df.ProductUid==v,"Texts"].iloc[0] for v in variants)

# DBSCAN Levenshtein

## Prepare X
X will contain a matrix containing a distance between products

In [75]:
df_keyword = df[df.Texts.str.lower().str.contains(keyword)]
len(df_keyword)

1291

In [76]:
df_keyword

,TaxonomyPath,TaxonomyCode,Texts,ProductUid
432,Plomberie/Tube et raccord/Canalisation PVC/Rac...,PL_5/1/4/2/1/,Manchon de dilatation simple mf d.80\nwww.mano...,501058910
433,Plomberie/Toiture et couverture/Gouttière et r...,PL_5/10/2/2/,Collier bride insert p.7/150 d.80 marron\nwww....,501058387
434,Plomberie/Toiture et couverture/Gouttière et r...,PL_5/10/2/2/,Coude mf 45' d.100 gris\nwww.manomano.fr\nCT4G...,501058449
435,Plomberie/Non catégorisé,PL_5/13/,Element de courette anglaise 130\nwww.manomano...,501058509
624,Plomberie/Arrosage/De surface,PL_5/12/1/,Boite de robinet maraicher rob.20/27\nwww.nico...,501057396
...,...,...,...,...
94518,Sanitaire/Pièces détachées,SA_6/11/,Dvab-embout de clapet pour reglage hauteur\n04...,501056782
95012,Plomberie/Évacuation/Vidage - Siphon - Bonde -...,PL_5/6/1/2/,D580-se siphon grille garde 50mm/580-600\nwww....,501056771
95013,Sanitaire/Pièces détachées,SA_6/11/,D690a-capot pr receveur 90 / abs chrome\nwww.m...,501056772
98076,Sanitaire/Équipement WC/Raccordement WC/Évacua...,SA_6/3/10/2/,Pipe wc sortie droite d.100 joint 85/107 vrac\...,501058980


In [80]:
df_keyword.sort_values(by="ProductUid",inplace=True)
df_keyword = df_keyword.reset_index()

In [81]:
df_keyword

,index,TaxonomyPath,TaxonomyCode,Texts,ProductUid
0,0,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 16\nPince à sertir TH 1...,500608091
1,1,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 20\nPince à sertir TH 2...,500608095
2,2,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 26\nPince à sertir TH 2...,500608097
3,3,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 32\nPince à sertir TH 3...,500608099
4,4,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 40\nPince à sertir TH 4...,500608101
...,...,...,...,...,...
1286,1286,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Akku-Press 14V ACC Basic-Pack\nwww.manoma...,501078513
1287,1287,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Power-Press ACC Basic-Pack\nwww.nicoll.fr...,501078522
1288,1288,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Mini-Press 14V ACC Basic-Pack\nwww.manoma...,501078524
1289,1289,Électricité/Sécurité et communication/Batterie...,EL_3/7/6/1/,"Batterie de substitution, PM/PMZ, Li-ion 3,2 A...",501103553


In [82]:
df[df.ProductUid==500608091]

,TaxonomyPath,TaxonomyCode,Texts,ProductUid
23861,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 16\nPince à sertir TH 1...,500608091


In [83]:
df_metrics = df_metrics.merge(df_keyword[["index","ProductUid"]],left_on="ProductUid_A",right_on="ProductUid")
df_metrics.drop(columns="ProductUid",inplace=True)
df_metrics.rename(columns={"index":"x1"},inplace=True)

df_metrics= df_metrics.merge(df_keyword[["index","ProductUid"]],left_on="ProductUid_B",right_on="ProductUid")
df_metrics.drop(columns="ProductUid",inplace=True)
df_metrics.rename(columns={"index":"x2"},inplace=True)
df_metrics

,Unnamed: 0,Rouge,In_A_not_B,In_B_not_A,ProductUid_A,ProductUid_B,variant,rank_levenshtein,Levenshtein,x1,x2
0,0,0.991561,"TF18, 0.081","0.046, TF188MF",501059169,501059172,"(501059169, 501059172)",6608,0.833333,1200,1203
1,728857,0.000000,not match,not match,501059150,501059172,"(501059150, 501059172)",249790,0.375000,1188,1203
2,2312,0.947146,"0.109, ME, 0.100, CLASSEMENT_EUROCLASSE, d.50,...","d’eau., E, 0.046, 0.040, Inventé, marché.<br>N...",501059211,501059172,"(501059172, 501059211)",3072,0.900000,1216,1203
3,591708,0.000000,not match,not match,501059160,501059172,"(501059160, 501059172)",153244,0.412698,1198,1203
4,34797,0.822270,"d’impuretés</li><li>Rayon, collecteurs, >, dim...","ou, rayon, eaux</li><li>Becquet, l'intérieur/É...",501056883,501059172,"(501056883, 501059172)",67137,0.491228,205,1203
...,...,...,...,...,...,...,...,...,...,...,...
832690,226009,0.000000,not match,not match,501058449,501057396,"(501057396, 501058449)",698123,0.271186,558,305
832691,224690,0.000000,not match,not match,501058387,501057396,"(501057396, 501058387)",101489,0.447368,498,305
832692,224620,0.000000,not match,not match,501058509,501057396,"(501057396, 501058509)",108741,0.441176,615,305
832693,227722,0.000000,not match,not match,501058910,501058449,"(501058449, 501058910)",391106,0.338983,1005,558


In [233]:
X_levenshtein = np.zeros([len(df_keyword),len(df_keyword)])
print(X_levenshtein.shape)
X_levenshtein

(1291, 1291)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [234]:
for id in df_metrics.index:
    i, j,val = df_metrics.loc[id,['x1','x2','Levenshtein']]
    X_levenshtein[i,j] = 1-val
    X_levenshtein[j,i] = 1-val

In [235]:
X_levenshtein

array([[0.        , 0.08      , 0.04      , ..., 0.76271186, 0.69863014,
        0.63380282],
       [0.08      , 0.        , 0.04      , ..., 0.76271186, 0.67123288,
        0.69014085],
       [0.04      , 0.04      , 0.        , ..., 0.76271186, 0.67123288,
        0.66197183],
       ...,
       [0.76271186, 0.76271186, 0.76271186, ..., 0.        , 0.80487805,
        0.75      ],
       [0.69863014, 0.67123288, 0.67123288, ..., 0.80487805, 0.        ,
        0.12765957],
       [0.63380282, 0.69014085, 0.66197183, ..., 0.75      , 0.12765957,
        0.        ]])

## DBSCAN

In [300]:
from sklearn.cluster import DBSCAN
clf=DBSCAN(eps=0.3,metric="precomputed",min_samples=2)

In [301]:
clf.fit(X_levenshtein)

DBSCAN(eps=0.3, metric='precomputed', min_samples=2)

In [302]:
clf.labels_.max() ,np.average(clf.labels_== - 1)

(100, 0.11541440743609604)

In [303]:
 visualize_variants(df,df_keyword.loc[clf.labels_==5,"ProductUid"])

("Manchon a sertir fluxo d.20 - per d.20\nwww.nicoll.fr\nFLM20PER20\nNICOLL\nSanitaire-Chauffage/Les solutions pour l'intérieur/Réseaux de distribution/Multicouche FLUXO®\nMANCHONS A SERTIR\nDésignation www.nicoll.fr: Manchon multicouche de transition à sertir PER\nAvantages: <h3>Les + Nicoll <span>performance</span></h3><ul><li>Diamètre de passage optimisé</li><li>Double joint torique</li></ul><h3>Les + Nicoll <span>installation</span></h3><ul><li>Fuite organisé permettant d'éviter les oublis de sertissage lors de la mise en pression du réseau</li><li>Encombrement optimisé pour faciliter la pose</li><li>Bague de sertissage sécable sécurisant le positionnement de la mâchoire de sertissage (TH)</li><li>Joint diélectrique dynamique confirmant la bonne insertion du tube</li><li>Force d'insertion optimale facilitant la mise en oeuvre</li></ul>\nDescription: <p>Les raccords à sertir de la gamme Fluxo® sont conçus\xa0pour vous permettre de réaliser rapidement tous les types d’installation, a

In [253]:
not_class = visualize_variants(df,df_keyword.loc[clf.labels_==-1,"ProductUid"])

In [254]:
print(len(not_class))
for i in not_class[:20]:
    print(i.split('\n')[0])

149
CINTREUSE POUR MULTICOUCHES
Corps caniveau mural faible hauteur+natte s= d.40
Siphon p/carrelage s/vert.50/63 g.10x10 inox
L212-bonde de lavabo automatique
Bmt01-tubulure lavabo bimatiere
L231-siphon de bidet 1 1/4 forme p
L260-bonde lavabo a bouchon-chainette
L268-tubulure.lavab.sortie arriere 1 1/4
L262-bde lave-main grille d.55 corps polypro
L263-bde lave-main polypro grille d=63
Trop plein souple annele d25 lg 355mm avec grille
Gs008-siphon groupe securite g.1x5/4  nf
L268m-tub.lavabo sort.arriere  metallisee
Cu219-combine urinoir   bonde et siphon
Bmt02 - ens.comp./bm211 + bmt01
B202-vid.baignoire chainette + siph.207
207-siphon baignoire horizontal 360'
050-plateau bouchon+chainette bonde 84
558-bde bouchon+chainette / evier inox
556-bde+trop plein evier inox


In [255]:
key = "bde lave-main"
df.loc[df.Texts.str.contains(key),:]

,TaxonomyPath,TaxonomyCode,Texts,ProductUid
42594,Plomberie/Évacuation/Vidage - Siphon - Bonde -...,PL_5/6/1/4/,L262-bde lave-main grille d.55 corps polypro\n...,501056546
82493,Plomberie/Évacuation/Vidage - Siphon - Bonde -...,PL_5/6/1/4/,L263-bde lave-main polypro grille d=63\nwww.ni...,501056547


In [256]:
for i in df.loc[df.Texts.str.contains(key),"Texts"]:
    print(i)
    print("*****************************************")


L262-bde lave-main grille d.55 corps polypro
www.nicoll.fr
0201031
NICOLL
Les solutions pour l'intérieur/Salle de bains/Vidages lavabos et bidets
BONDES LAVABOS SAS
Désignation www.tereva-direct.fr: Bonde lavabo à grille L262
Désignation rexel.fr: L262-bde lave-main grille d.55 corps polypro
Désignation www.nicoll.fr: Bonde de lavabo plastique à grille
Sous-titre: Bonde de lavabo plastique à grille
Avantages: <h3>Les + Nicoll <span>performance</span></h3><ul><li>Etanchéité normalisée</li><li>Design</li></ul><h3>Les + Nicoll <span>installation</span></h3><ul><li>Serrage de 1 à 50mm</li><li>Encombrement réduit</li></ul>
Description: <p>Bonde de lavabo corps PP sortie 1 1/4"</p><p>Grille Inox D55mm, D60mm ou D63mm</p><p>Grille Inox plate (D55mm et D63mm) ou bombée (D60mm)</p>
ATTRIBUTES: Hauteur : 0.040 m | Largeur : 0.070 m Profondeur : 0.110 m | Poids : 0.055 kg | Volume : 3.080 m3 |
Hauteur: 0.040 m
Largeur: 0.070 m Profondeur
Poids: 0.055 kg
Volume: 3.080 m3
Type / Application: Lavabo

In [257]:
p = [501056546,501056547]
df_metrics.loc[(df_metrics.ProductUid_B.isin(p))&(df_metrics.ProductUid_A.isin(p)),"Levenshtein"]

150438    0.634146
Name: Levenshtein, dtype: float64

# DBSCAN Rouge  
This sections needs to be ran after the Levenshtein

In [258]:
X_rouge=np.zeros([len(df_keyword),len(df_keyword)])
for id in df_metrics.index:
    i, j,val = df_metrics.loc[id,['x1','x2','Rouge']]
    X_rouge[i,j] = 1-val
    X_rouge[j,i] = 1-val

In [270]:
clf_rouge=DBSCAN(eps=0.2,metric="precomputed",min_samples=2)
clf_rouge.fit(X_rouge)
clf_rouge.labels_.max(), np.average(clf_rouge.labels_== - 1)

(110, 0.11463981409759876)

In [305]:
visualize_variants(df,df_keyword.loc[clf_rouge.labels_==100,"ProductUid"])

("Caniveau douche 990mm d40 + grille reversible\nwww.nicoll.fr\nSICAN100R\nNICOLL\nLes solutions pour l'intérieur/Salle de bains/Douches à l'italienne DOCIA®\nCANIVEAUX DOUCHE\nDésignation www.nicoll.fr: Caniveau de douche pour grille 50 mm avec grille réversible\nSous-titre: Caniveaux DOCIA : la solution Design pour douche carrelée de plain-pied\nAvantages: <h3>Les + Nicoll <span>performance</span></h3><ul><li>Garantit l'étanchéité de votre installation</li><li>Grand débit d'évacuation 40L/min</li><li>Garde d'eau de 50 mm, système anti-remontées d'odeurs</li><li>Hygiène et facilité d'entretien, siphon extractible</li><li>Grille réversible, finition inox pleine et/ou à carreler</li></ul><h3>Les + Nicoll <span>installation</span></h3><ul><li>Réglage en hauteur du corps du caniveau</li><li>Réglage en hauteur du cadre avec système de cales</li><li>Système de bridage de la natte d'étanchéité</li><li>Platine de largeur réduite pour une pose contre-cloison</li><li>Sortie à coller </li></ul>\

In [279]:
not_class = visualize_variants(df,df_keyword.loc[clf_rouge.labels_==-1,"ProductUid"])
print(len(not_class))
for i in not_class[:20]:
    print(i.split('\n')[0])

148
Pipe courte wc 90' d.100f a joint 85/107 vrac
Manchon a sertir fluxo d.20 - per d.20
Grille inox brosse pour sicanatm/sicanatmfh
Corps caniveau mural faible hauteur+natte s= d.40
Pipe orientable  d.100 joint 85/107 recoupable
Siphon p/carrelage s/vert.50/63 g.10x10 inox
L212-bonde de lavabo automatique
Bmt01-tubulure lavabo bimatiere
L260-bonde lavabo a bouchon-chainette
L268-tubulure.lavab.sortie arriere 1 1/4
L262-bde lave-main grille d.55 corps polypro
L263-bde lave-main polypro grille d=63
Gs008-siphon groupe securite g.1x5/4  nf
L268m-tub.lavabo sort.arriere  metallisee
Cu219-combine urinoir   bonde et siphon
B202-vid.baignoire chainette + siph.207
207-siphon baignoire horizontal 360'
Eb07-easybain wave cable 650mm
Eb17-easybain wave cable 1m
552-siphon evier tube reglable g.1 1/2


# Combined DBSCAN

In [274]:
X_combined = (X_levenshtein+X_rouge)*0.5

In [306]:
clf_combined=DBSCAN(eps=0.25,metric="precomputed",min_samples=2)
clf_combined.fit(X_rouge)
clf_combined.labels_.max(), np.sum(clf_combined.labels_== - 1)

(100, 125)

In [296]:
visualize_variants(df,df_keyword.loc[clf_combined.labels_==2,"ProductUid"])

("42153-ensemble sur barre inox chrome d25 750mm- river 3 jets - flex chrome 1.75\nwww.nicoll.fr\n0342153\nNICOLL\nLes solutions pour l'intérieur/Salle de bains/Hydrothérapie HYDROSENSO®\nENS DOUCHE  -  BARRE SAS\nCaractéristiques: <ul>\n<li>Barre : Inox brut diamètre 25 mm</li>\n<li>Longueur : 650 mm</li>\n<li>Accessoire : Porte savon</li>\n<li>Flexible : métallo-plastique 1,75mm</li>\n<li>Douche : Diamètre 83 mm</li>\n<li>ø : 83 mm</li>\n<li>Entraxe : 555 mm</li>\n<li>Jet : Jet de pluie Jet de massage Jet de mousse</li>\n<li>Nombre de jets : 3</li>\n<li>Sous coque : Sans</li>\n<li>Supplément douche : Anticalcaire Entraxe réglable Porte savon</li>\n<li>Design flexible : Double agrafage</li>\n<li>Type d'assemblage : A visser</li>\n<li>Débit : 10.3</li>\n<li>Unité du débit : L/min</li>\n<li>Matière : Laiton ABS Metal plastique</li>\n<li>Finition : Chromé</li>\n<li>ACS – Contact Eau Potable : Avec</li>\n</ul>\n\nDésignation www.nicoll.fr: Ensemble sur barre River 3 jets\nSous-titre: Ense

In [280]:
not_class = visualize_variants(df,df_keyword.loc[clf_combined.labels_==-1,"ProductUid"])
print(len(not_class))
for i in not_class[:20]:
    print(i.split('\n')[0])

125
Pipe courte wc 90' d.100f a joint 85/107 vrac
Manchon a sertir fluxo d.20 - per d.20
Grille inox brosse pour sicanatm/sicanatmfh
Corps caniveau mural faible hauteur+natte s= d.40
Pipe orientable  d.100 joint 85/107 recoupable
Siphon p/carrelage s/vert.50/63 g.10x10 inox
L212-bonde de lavabo automatique
Bmt01-tubulure lavabo bimatiere
L268-tubulure.lavab.sortie arriere 1 1/4
L262-bde lave-main grille d.55 corps polypro
Gs008-siphon groupe securite g.1x5/4  nf
L268m-tub.lavabo sort.arriere  metallisee
Cu219-combine urinoir   bonde et siphon
B202-vid.baignoire chainette + siph.207
207-siphon baignoire horizontal 360'
Eb07-easybain wave cable 650mm
Eb17-easybain wave cable 1m
050-plateau bouchon+chainette bonde 84
C5907-vid.complet av siphon ev.gres 2bacs
Tbs02-bonde turbosol /film 150x180+colle


In [294]:
key = "easybain"
df.loc[df.Texts.str.contains(key),:]

,TaxonomyPath,TaxonomyCode,Texts,ProductUid
1199,Sanitaire/Pièces détachées,SA_6/11/,Eb07-easybain wave cable 650mm\nrexel.fr\n0203...,501056561
82495,Plomberie/Évacuation/Vidage - Siphon - Bonde -...,PL_5/6/1/1/,Eb17-easybain wave cable 1m\nwww.nicoll.fr\n02...,501056562


In [284]:
for i in df.loc[df.Texts.str.contains(key),"Texts"]:
    print(i)
    print("*****************************************")

Eb07-easybain wave cable 650mm
rexel.fr
0203611
NICOLL
Sanitaire/Accessoires sanitaire/Vidage
VIDAGES BAIGNOIRE SAS
Désignation www.nicoll.fr: Vidage baignoire Easybain
Sous-titre: Une performance et un confort de pose inégalés
Avantages: <h3>Les + Nicoll <span>performance</span></h3><ul><li>Débit de 70l/min</li><li>Anti encrassement</li><li>Préhension optimisée pour une ouverture sans effort</li><li>Design du volant de commande en laiton ou ABS</li><li>Garde d'eau normalisée de 50mm</li><li>Certifié NF et breveté</li></ul><h3>Les + Nicoll <span>installation</span></h3><ul><li>Siphon 100% bi-injecté</li><li>Encombrement sous baignoire réduit à 94mm</li><li>Rotule orientable à 360°</li><li>Rotule inclinable à +/- 5°</li><li>Trop plein pré-monté ajustable en hauteur</li></ul>
Désignation rexel.fr: Eb07-easybain wave cable 650mm
Description:           VIDAGE EASYBAIN WAVE CABLE 650MM
ATTRIBUTES: Hauteur : 0.070 m | Largeur : 0.430 m Profondeur : 0.190 m | Poids : 0.445 kg | Volume : 57.19

In [288]:
p = df.loc[df.Texts.str.contains(key),"ProductUid"]
df_metrics.loc[(df_metrics.ProductUid_B.isin(p))&(df_metrics.ProductUid_A.isin(p)),["Levenshtein","Rouge"]]

,Levenshtein,Rouge
153588,0.877193,0.0


In [286]:
df.loc[df.Texts.str.contains(key),"ProductUid"]

1199     501056561
82495    501056562
Name: ProductUid, dtype: int64

# Export Christophe

In [310]:
df_keyword["cluster"] = clf_combined.labels_

In [311]:
df_keyword

,index,TaxonomyPath,TaxonomyCode,Texts,ProductUid,cluster
0,0,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 16\nPince à sertir TH 1...,500608091,0
1,1,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 20\nPince à sertir TH 2...,500608095,0
2,2,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 26\nPince à sertir TH 2...,500608097,0
3,3,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 32\nPince à sertir TH 3...,500608099,0
4,4,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/1/,Pince à sertir REMS TH 40\nPince à sertir TH 4...,500608101,0
...,...,...,...,...,...,...
1286,1286,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Akku-Press 14V ACC Basic-Pack\nwww.manoma...,501078513,-1
1287,1287,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Power-Press ACC Basic-Pack\nwww.nicoll.fr...,501078522,-1
1288,1288,Outillage - Droguerie - Quincaillerie/Outillag...,OU_4/7/1/1/2/,REMS Mini-Press 14V ACC Basic-Pack\nwww.manoma...,501078524,-1
1289,1289,Électricité/Sécurité et communication/Batterie...,EL_3/7/6/1/,"Batterie de substitution, PM/PMZ, Li-ion 3,2 A...",501103553,-1


In [313]:
df_keyword.to_excel("DBSCAN_Cluster_Nicoll.xlsx")